# Getting started

This tutorial uses safeds on titanic passenger data to predict who will survive and who will not.


1. Load your data into a `Table`, the data is available under `docs/tutorials/data/titanic.csv`:


In [4]:
from safeds.data.tabular.containers import Table

titanic = Table.from_csv_file("data/titanic.csv")

2. Use `OneHotEncoder` to create an encoder, that will be used later to transform the titanic table.
Use the `fit` function of the `OneHotEncoder` to pass the table and the column names, that will be used as features to predict who will survive to the encoder:


In [13]:
from safeds.data.tabular.transformation import OneHotEncoder
encoder = OneHotEncoder().fit(titanic, ["sex", "travel_class"])

3. Transform the table using the fitted encoder:


In [14]:
transformed_table = encoder.transform(titanic)
